In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2024-04-26 07:32:05.198230: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 07:32:05.198391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 07:32:05.362280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_set=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_set= pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
sub_set=pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [5]:
train_set.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
test_set.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [7]:
sub_set.head(5)

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [8]:
print("subset number :", sub_set.shape)
print("testset number :", test_set.shape)

subset number : (4277, 2)
testset number : (4277, 13)


In [9]:
train_set_final=train_set.drop('PassengerId', axis=1)

In [10]:
train_set_final.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [12]:
train_set_obj= train_set_final.select_dtypes(include=['object'])

In [13]:
train_set_obj.dtypes

HomePlanet     object
CryoSleep      object
Cabin          object
Destination    object
VIP            object
Name           object
dtype: object

In [14]:
train_set_obj.head(10)

,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
0,Europa,False,B/0/P,TRAPPIST-1e,False,Maham Ofracculy
1,Earth,False,F/0/S,TRAPPIST-1e,False,Juanna Vines
2,Europa,False,A/0/S,TRAPPIST-1e,True,Altark Susent
3,Europa,False,A/0/S,TRAPPIST-1e,False,Solam Susent
4,Earth,False,F/1/S,TRAPPIST-1e,False,Willy Santantines
5,Earth,False,F/0/P,PSO J318.5-22,False,Sandie Hinetthews
6,Earth,False,F/2/S,TRAPPIST-1e,False,Billex Jacostaffey
7,Earth,True,G/0/S,TRAPPIST-1e,False,Candra Jacostaffey
8,Earth,False,F/3/S,TRAPPIST-1e,False,Andona Beston
9,Europa,True,B/1/P,55 Cancri e,False,Erraiam Flatic


In [49]:
total_missing = train_set_preprocessed.isnull().sum().sum()
print("Total missing values in the dataset:", total_missing)

# Number of missing values in each column
missing_per_column = train_set_preprocessed.isnull().sum()
print("Number of missing values in each column:")
print(missing_per_column)

Total missing values in the dataset: 1122
Number of missing values in each column:
HomePlanet        0
CryoSleep         0
Destination       0
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
dtype: int64


In [26]:
import pandas as pd

# Supposons que 'df' soit votre DataFrame
train_set_obj_2=train_set_obj
# Parcourir toutes les colonnes du DataFrame
for colonne in train_set_obj_2.columns:
    # Vérifier si le type de données de la colonne est un objet
    if train_set_obj_2[colonne].dtype == 'object':
        # Calculer le nombre d'objets uniques dans la colonne
        nombre_objets_uniques = train_set_obj_2[colonne].nunique()
            
        # Afficher le nombre d'objets uniques dans la colonne
        print("Nombre d'objets uniques dans la colonne '{}' : {}".format(colonne, nombre_objets_uniques))

Nombre d'objets uniques dans la colonne 'HomePlanet' : 3
Nombre d'objets uniques dans la colonne 'CryoSleep' : 2
Nombre d'objets uniques dans la colonne 'Destination' : 3
Nombre d'objets uniques dans la colonne 'VIP' : 2


In [19]:
train_set_final=train_set_final.drop('Name', axis=1)
train_set_obj= train_set_final.select_dtypes(include=['object'])

In [25]:
train_set_obj.head()

,HomePlanet,CryoSleep,Destination,VIP
0,Europa,False,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,False,TRAPPIST-1e,True
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False


In [31]:
nombre_objets_uniques = train_set_obj_2['HomePlanet'].nunique()

In [42]:
def preprocess_data(df):
    # Remplacer les valeurs manquantes par les valeurs les plus fréquentes
    for column in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']:
        count_values = df[column].value_counts()
        valeur_plus_frequente = count_values.idxmax()
        df[column].fillna(valeur_plus_frequente, inplace=True)

    # Remplacer les valeurs catégorielles par des entiers
    df['HomePlanet'].replace({'Europa': 0, 'Earth': 1, 'Mars': 2}, inplace=True)
    df['CryoSleep'].replace({'False': 0, 'True': 1}, inplace=True)
    df['Destination'].replace({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e': 2}, inplace=True)
    df['VIP'].replace({'False': 0, 'True': 1}, inplace=True)

    # Convertir les colonnes en int64
    df['HomePlanet'] = df['HomePlanet'].astype('int64')
    df['CryoSleep'] = df['CryoSleep'].astype('int64')
    df['Destination'] = df['Destination'].astype('int64')
    df['VIP'] = df['VIP'].astype('int64')

    return df

# Appeler la fonction preprocess_data avec train_set_obj_2 comme paramètre
train_set_obj_2 = preprocess_data(train_set_obj_2)


/tmp/ipykernel_33/3877942590.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(valeur_plus_frequente, inplace=True)
/tmp/ipykernel_33/3877942590.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [43]:
train_set_preprocessed=preprocess_data(train_set_final)

/tmp/ipykernel_33/3877942590.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(valeur_plus_frequente, inplace=True)
/tmp/ipykernel_33/3877942590.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [47]:
y.head(5)

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

In [46]:
y = train_set_preprocessed['Transported']
train_set_preprocessed=train_set_preprocessed.drop('Transported', axis=1)

In [50]:
# Calculer la moyenne de chaque colonne
moyennes = train_set_preprocessed.mean()

# Remplacer les valeurs NaN par la moyenne de chaque colonne
train_set_preprocessed = train_set_preprocessed.fillna(moyennes)

train_set_preprocessed = train_set_preprocessed.astype('int64')

In [54]:
X=train_set_preprocessed

# # **Regression**

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser le modèle de régression logistique
model = LogisticRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle:", accuracy)


Précision du modèle: 0.7809085681426107


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# # decision trees****

In [56]:
from sklearn.tree import DecisionTreeClassifier

# Initialiser le modèle d'arbre de décision
model = DecisionTreeClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (Arbres de décision):", accuracy)


Précision du modèle (Arbres de décision): 0.7170787809085681


# # Random forest****

In [57]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser le modèle de forêts aléatoires
model = RandomForestClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (Forêts aléatoires):", accuracy)


Précision du modèle (Forêts aléatoires): 0.7705577918343876


# # SVM****

In [58]:
from sklearn.svm import SVC

# Initialiser le modèle SVM
model = SVC()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (SVM):", accuracy)


Précision du modèle (SVM): 0.7648073605520413


# # KNN****

In [59]:
from sklearn.neighbors import KNeighborsClassifier

# Initialiser le modèle k-NN
model = KNeighborsClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (k-NN):", accuracy)


Précision du modèle (k-NN): 0.7567567567567568


# # gradient boosting****

In [60]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialiser le modèle de gradient boosting
model = GradientBoostingClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (Gradient Boosting):", accuracy)


Précision du modèle (Gradient Boosting): 0.7797584818861415


# # AdaBoost****

In [63]:
from sklearn.ensemble import AdaBoostClassifier

# Initialiser le modèle AdaBoost
model = AdaBoostClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, predictions)
print("Précision du modèle (AdaBoost):", accuracy)


Précision du modèle (AdaBoost): 0.7791834387579069


# # best model for the moment in machine learning is linear regression with an accuracy of 0.78 (I will optemize it later and I will try to use deep learning********)